## Running a simulator using existing data
Consider the case when input data already exists, and that data already has a causal structure.
We would like to simulate treatment assignment and outcomes based on this data.

### Initialize the data
First we load the desired data into a pandas DataFrame:

In [1]:
import pandas as pd
from causallib.datasets import load_nhefs # nhefs with categorical outcome
from causallib.datasets import load_nhefs_survival # nhefs with survival outcome
from causallib.datasets import CausalSimulator
from causallib.datasets import generate_random_topology

In [2]:
data = load_nhefs()
X_given = data.X

say we want to create three more variables: covariate, treatment and outcome.
This will be a bit difficult to hardwire a graph with many variables, so lets use the random topology generator:

In [3]:
topology, var_types = generate_random_topology(n_covariates=1, p=0.4,
                                               n_treatments=1, n_outcomes=1,
                                               given_vars=X_given.columns)

In [4]:
topology

,age,race,sex,smokeintensity,smokeyrs,wt71,active_1,active_2,education_2,education_3,...,education_5,exercise_1,exercise_2,age^2,wt71^2,smokeintensity^2,smokeyrs^2,18,19,20
age,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
race,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
sex,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
smokeintensity,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
smokeyrs,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
wt71,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
active_1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
active_2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
education_2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
education_3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Now we create the simulator based on the variables topology:

In [5]:
outcome_types = "categorical"
link_types = ['linear'] * len(var_types)
prob_categories = pd.Series(data=[[0.5, 0.5] if typ in ["treatment", "outcome"] else None for typ in var_types],
                            index=var_types.index)
treatment_methods = "gaussian"
snr = 0.9
treatment_importance = 0.8
effect_sizes = None
sim = CausalSimulator(topology=topology.values, prob_categories=prob_categories,
                      link_types=link_types, snr=snr, var_types=var_types,
                      treatment_importances=treatment_importance,
                      outcome_types=outcome_types,
                      treatment_methods=treatment_methods,
                      effect_sizes=effect_sizes)

Now in order to generate data based on the given data we need to specify:

In [6]:
X, prop, y = sim.generate_data(X_given=X_given)

C:\Users\lucp14006\CML_Codes\Practice Code\causallib\causallib\simulation\CausalSimulator3.py:447: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False False False ... False  True  True]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X.loc[:, X_given.columns] = X_given
C:\Users\lucp14006\CML_Codes\Practice Code\causallib\causallib\simulation\CausalSimulator3.py:447: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False False False ... False False False]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X.loc[:, X_given.columns] = X_given
C:\Users\lucp14006\CML_Codes\Practice Code\causallib\causallib\simulation\CausalSimulator3.py:447: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False  T

In [8]:
prop.head()

19     
     0    1
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  1.0  0.0
4  0.0  1.0

In [9]:
X.head()

,age,race,sex,smokeintensity,smokeyrs,wt71,active_1,active_2,education_2,education_3,...,education_5,exercise_1,exercise_2,age^2,wt71^2,smokeintensity^2,smokeyrs^2,18,19,20
0,42.0,1.0,0.0,30.0,29.0,79.04,False,False,False,False,...,False,False,True,1764.0,6247.3216,900.0,841.0,-305.190882,1.0,0.0
1,36.0,0.0,0.0,20.0,24.0,58.63,False,False,True,False,...,False,False,False,1296.0,3437.4769,400.0,576.0,-78.951842,0.0,0.0
2,56.0,1.0,1.0,20.0,26.0,56.81,False,False,True,False,...,False,False,True,3136.0,3227.3761,400.0,676.0,442.292795,0.0,1.0
3,68.0,1.0,0.0,3.0,53.0,59.42,True,False,False,False,...,False,False,True,4624.0,3530.7364,9.0,2809.0,798.250832,0.0,1.0
4,40.0,0.0,0.0,20.0,19.0,87.09,True,False,True,False,...,False,True,False,1600.0,7584.6681,400.0,361.0,-521.029546,1.0,0.0


### Format the data for training and save it

Now that we generated some data, we can format it so it would be easier to train and validate:

In [7]:
observed_set, validation_set = sim.format_for_training(X, prop, y)

In [10]:
observed_set.head()

,x_age,x_race,x_sex,x_smokeintensity,x_smokeyrs,x_wt71,x_active_1,x_active_2,x_education_2,x_education_3,...,x_education_5,x_exercise_1,x_exercise_2,x_age^2,x_wt71^2,x_smokeintensity^2,x_smokeyrs^2,x_18,t_19,y_20
0,42.0,1.0,0.0,30.0,29.0,79.04,False,False,False,False,...,False,False,True,1764.0,6247.3216,900.0,841.0,-305.190882,1.0,0.0
1,36.0,0.0,0.0,20.0,24.0,58.63,False,False,True,False,...,False,False,False,1296.0,3437.4769,400.0,576.0,-78.951842,0.0,0.0
2,56.0,1.0,1.0,20.0,26.0,56.81,False,False,True,False,...,False,False,True,3136.0,3227.3761,400.0,676.0,442.292795,0.0,1.0
3,68.0,1.0,0.0,3.0,53.0,59.42,True,False,False,False,...,False,False,True,4624.0,3530.7364,9.0,2809.0,798.250832,0.0,1.0
4,40.0,0.0,0.0,20.0,19.0,87.09,True,False,True,False,...,False,True,False,1600.0,7584.6681,400.0,361.0,-521.029546,1.0,0.0


In [ ]:
print(X.columns)
print(observed_set.columns)
print(X.equals(X_given)) # checks if generated X is identical to X_given

Index([             'age',             'race',              'sex',
         'smokeintensity',         'smokeyrs',             'wt71',
               'active_1',         'active_2',      'education_2',
            'education_3',      'education_4',      'education_5',
             'exercise_1',       'exercise_2',            'age^2',
                 'wt71^2', 'smokeintensity^2',       'smokeyrs^2',
                       18,                 19,                 20],
      dtype='object')
Index(['x_age', 'x_race', 'x_sex', 'x_smokeintensity', 'x_smokeyrs', 'x_wt71',
       'x_active_1', 'x_active_2', 'x_education_2', 'x_education_3',
       'x_education_4', 'x_education_5', 'x_exercise_1', 'x_exercise_2',
       'x_age^2', 'x_wt71^2', 'x_smokeintensity^2', 'x_smokeyrs^2', 'x_18',
       't_19', 'y_20'],
      dtype='object')
False


In [16]:
X.shape, observed_set.shape, validation_set.shape

((1566, 21), (1566, 21), (1566, 5))

observed_set is the observed dataset (excluding hidden variables)validation_set is for validation purposes - it has the counterfactuals, the treatments assignment and the propensity for every sample.
You can save the datasets into csv:

In [17]:
covariates = observed_set.loc[:, observed_set.columns.str.startswith("x_")]
print(covariates.shape)
covariates.head()

(1566, 19)


,x_age,x_race,x_sex,x_smokeintensity,x_smokeyrs,x_wt71,x_active_1,x_active_2,x_education_2,x_education_3,x_education_4,x_education_5,x_exercise_1,x_exercise_2,x_age^2,x_wt71^2,x_smokeintensity^2,x_smokeyrs^2,x_18
0,42.0,1.0,0.0,30.0,29.0,79.04,False,False,False,False,False,False,False,True,1764.0,6247.3216,900.0,841.0,-305.190882
1,36.0,0.0,0.0,20.0,24.0,58.63,False,False,True,False,False,False,False,False,1296.0,3437.4769,400.0,576.0,-78.951842
2,56.0,1.0,1.0,20.0,26.0,56.81,False,False,True,False,False,False,False,True,3136.0,3227.3761,400.0,676.0,442.292795
3,68.0,1.0,0.0,3.0,53.0,59.42,True,False,False,False,False,False,False,True,4624.0,3530.7364,9.0,2809.0,798.250832
4,40.0,0.0,0.0,20.0,19.0,87.09,True,False,True,False,False,False,True,False,1600.0,7584.6681,400.0,361.0,-521.029546


In [22]:
treatment_outcome = observed_set.loc[:, (observed_set.columns.str.startswith("t_") |
                                         observed_set.columns.str.startswith("y_"))]
print(treatment_outcome.shape)
treatment_outcome.head()

(1566, 2)


,t_19,y_20
0,0,0
1,0,1
2,0,1
3,1,1
4,1,0


In [23]:
print(validation_set.shape)
validation_set.head()

(1566, 5)


,t_19,p_19_0,p_19_1,cf_20_0,cf_20_1
0,0,1.0,0.0,0,0
1,0,1.0,0.0,1,1
2,0,1.0,0.0,1,1
3,1,1.0,0.0,1,1
4,1,1.0,0.0,0,0


checking if the values of y in the factuals file and cfs are exactly the same

In [23]:
df_compare = pd.DataFrame({
    'factual_y': observed_set['y_20'], 
    'cf_y0': validation_set['cf_20_0'],
    'cf_y1': validation_set['cf_20_1']
})

df_compare.tail(15)


,factual_y,cf_y0,cf_y1
1613,0.0,0,0
1614,0.0,0,0
1615,0.0,0,0
1616,1.0,1,1
1617,1.0,1,1
1618,0.0,0,0
1619,1.0,1,1
1620,0.0,0,0
1621,1.0,1,1
1622,0.0,0,0


In [ ]:
# (df_compare["factual_y"].equals(df_compare["cf_y0"]) and 
#  df_compare["cf_y0"].equals(df_compare["cf_y1"]))

mask = (df_compare["factual_y"] == df_compare["cf_y0"]) & (df_compare["cf_y0"] == df_compare["cf_y1"])
df_compare[~mask]

,factual_y,cf_y0,cf_y1
271,1.0,1,0
398,0.0,1,0
468,0.0,1,0
482,0.0,1,0
488,1.0,1,0
497,0.0,1,0
565,0.0,1,0
596,0.0,1,0
797,1.0,1,0
874,1.0,1,0


Only 24 out of ~1.5k differ ***

- maybe a linear link is not the ideal choice for 'how the covariates combine to generate a variable'
- for the linear link, the $\beta$(s) were not provided, therefore the values were randomly sampled from a normal distribution - this may not be a suitable approach for this data
- the graph was randomly generated

### Simulation for survival outcome

In [ ]:
survival_data = load_nhefs_survival()
X_given = survival_data.X
# survival_data.y.head()
# X_given.head()

0    0
1    0
2    0
3    1
4    0
Name: death, dtype: int64

In [3]:
topology, var_types = generate_random_topology(n_covariates=1, p=0.6,
                                               n_treatments=1, n_outcomes=1,
                                               given_vars=X_given.columns)

In [ ]:
outcome_types = "survival"
link_types = ['linear'] * len(var_types)
prob_categories = pd.Series(data=[[0.5, 0.5] if typ in ["treatment", "outcome"] else None for typ in var_types],
                            index=var_types.index)
treatment_methods = "gaussian"
snr = 0.9
treatment_importance = 0.8
effect_sizes = None
sim_survival1 = CausalSimulator(topology=topology.values, prob_categories=prob_categories,
                      link_types=link_types, snr=snr, var_types=var_types,
                      treatment_importances=treatment_importance,
                      outcome_types=outcome_types,
                      treatment_methods=treatment_methods,
                      effect_sizes=effect_sizes)

In [6]:
X, prop, y = sim_survival1.generate_data(X_given=X_given)

C:\Users\lucp14006\CML_Codes\Practice Code\causallib\causallib\simulation\CausalSimulator3.py:447: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False False False ... False  True  True]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X.loc[:, X_given.columns] = X_given
C:\Users\lucp14006\CML_Codes\Practice Code\causallib\causallib\simulation\CausalSimulator3.py:447: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False False False ... False False False]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X.loc[:, X_given.columns] = X_given
C:\Users\lucp14006\CML_Codes\Practice Code\causallib\causallib\simulation\CausalSimulator3.py:447: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False  T

In [9]:
observed_set, validation_set = sim.format_for_training(X, prop, y)

In [14]:
covariates = observed_set.loc[:, observed_set.columns.str.startswith("x_")]

treatment_outcome = observed_set.loc[:, (observed_set.columns.str.startswith("t_") |
                                         observed_set.columns.str.startswith("y_"))]

treatment_outcome = observed_set.loc[:, (observed_set.columns.str.startswith("t_") |
                                         observed_set.columns.str.startswith("y_"))]

df_compare = pd.DataFrame({
    'factual_y': observed_set['y_20'], 
    'cf_y0': validation_set['cf_20_0'],
    'cf_y1': validation_set['cf_20_1']
})  

mask = (df_compare["factual_y"] == df_compare["cf_y0"]) & (df_compare["cf_y0"] == df_compare["cf_y1"])
df_compare[~mask]

,factual_y,cf_y0,cf_y1
0,0.133953,0.133953,0.145281
1,0.891552,0.822033,0.891552
2,1.383597,1.383597,1.500607
3,145.710565,145.710565,158.033206
4,5.431786,5.008243,5.431786
...,...,...,...
1624,0.475595,0.475595,0.515816
1625,5.453227,5.453227,5.914402
1626,0.915026,0.915026,0.992409
1627,947.499651,947.499651,1027.629037


In this case, all the records in the simulations have different values.